## Тестовое задание Контур
#### Для выполнения задания я выполнил следующие шаги:
- Считать тренировочный и тестовый датасет
- Преобразовать тренировочный датасет в нужный вид для тренировки NER в spacy
- Преобразовать датасет нового вида в train.spacy файл
- Создать config.cfg файл с настройками для тренировки NER модели
- Запустить тренировку модели и сохранить ее в папку /output/model-best
- Используя модель добавить колонку 'extracted_part' в тестовый датасет в нужном формате

#### Считываем тренировочный датасет в pandas dataframe (Для тестирования измените путь до train.json)

In [74]:
import pandas as pd
import numpy as np

# Open json file using pandas:
df = pd.read_json('/Users/damirabdulaev/Downloads/nlp_test_task_2023/dataset/train.json')
df

,id,text,label,extracted_part
0,809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
1,854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,{'text': ['Поставщик должен предоставить обесп...
2,4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
3,184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
4,211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
...,...,...,...,...
1794,938335285,"«УТВЕРЖДАЮ» Директор МУП ""ПКП и ОРТ"" _________...",обеспечение гарантийных обязательств,{'text': ['Заказчиком установлено требование о...
1795,214609521,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."
1796,94811748,ДОГОВОР ПОДРЯДА № _____ «___»___________20____...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."
1797,903507299,ДОКУМЕНТАЦИЯ О ЗАКУПКЕ ЗАПРОС ПРЕДЛОЖЕНИЙ В ЭЛ...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."


#### Просмотреть информацию с колонок label, text и extracted_part

In [95]:
df['label'][0]

'обеспечение исполнения контракта'

In [96]:
df['text'][0]

'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета в соответствии с п.16 ч. 1

In [97]:
df['extracted_part'][0]

{'text': ['Размер обеспечения исполнения контракта 6593.25 Российский рубль'],
 'answer_start': [1279],
 'answer_end': [1343]}

 #### Привести дату в вид для тренировки NER в spacy. Вид такой: ('text', {'entities': [(start, end, 'part_of_text')]})

In [77]:
# Define the column that contains the extracted part data
extracted_part_column = 'extracted_part'
label_column = 'label'

# Define the function to convert extracted_part to ner training data format
def convert_to_ner_data(row):
    text = row['text']
    if text == '':
        text = ' '
    start = row['extracted_part']['answer_start'][0]
    end = row['extracted_part']['answer_end'][0]
    label = row[label_column]
    entities = [(start, end, label)]
    return (text, {'entities': entities})

# Convert the extracted_part column to ner training data format
train_data = []
for i, row in df.iterrows():
    train_data.append(convert_to_ner_data(row))

train_data[0]

('Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета в соответствии с п.16 ч. 

#### Для тренировки я использую модель ru_core_news_lg, но ее NER я буду тренировать самостоятельно. В коде я преобразую нашу тестовую дату в .spacy файл, который потом использую для тренировки модели

In [8]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp = spacy.load("ru_core_news_lg") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(train_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

os.chdir('/Users/damirabdulaev/Desktop/Project')
db.to_disk("./train.spacy") # save the docbin object

 39%|███▊      | 695/1799 [00:01<00:01, 760.51it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 52%|█████▏    | 933/1799 [00:01<00:01, 761.35it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 61%|██████    | 1093/1799 [00:01<00:00, 778.39it/s]

...

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 90%|█████████ | 1627/1799 [00:02<00:00, 820.55it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping

100%|██████████| 1799/1799 [00:02<00:00, 757.64it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


#### После преобразования информации в нужный вид и формат, нам нужно создать .cfg файл. Для этого зайдем на сайт:
https://spacy.io/usage/training#config
#### и используем генератор конфига для тренировки NER. Используем команду
python -m spacy init fill-config base_config.cfg config.cfg для создания config.cfg из base_config.cfg
#### который скачали ранее. Команда создаст модель, которую мы далее используем для нашей задачи.


#### После тренировки я получил такие результаты для лучшей (или почти лучшей) модели:
E   5
\#     10400
LOSS TOK2VEC  73101.68
LOSS NER  229.45
ENTS_F  76.58
ENTS_P  73.26
ENTS_R   80.23
SCORE    0.77



In [85]:
nlp = spacy.load('/Users/damirabdulaev/Desktop/Project/output/model-best')

In [78]:
counter = 0
for i in range(1799):
    doc = nlp(df['text'][i])
    entity_names = []
    for ent in doc.ents:
        entity_names.append(ent.text)

    # Print the extracted entity names
    if len(entity_names) == 0:
        if  df['extracted_part'][i]['text'][0] == "":
            counter += 1
        # print("HERE:")
        # print(df['extracted_part'][i]['text'][0])
        # print("------------------------------------------------")
    else:
        if entity_names[0] == df['extracted_part'][i]['text'][0]:
            counter += 1
        else:
            print(entity_names[0])
            print("----------------")
            print(df['extracted_part'][i]['text'][0])
            print("------------------------------------------------")
print(counter / 1799)

Размер обеспечения исполнения контракта устанавливается в размере 5 (пять) процентов от цены, по которой заключается контракт и составляет _________ (____________) рубля __ копейки.
----------------
Размер обеспечения исполнения контракта устанавливается в размере 5 (пять) процентов от цены, по которой заключается контракт и составляет ___
------------------------------------------------
Размер обеспечения гарантийных обязательств составляет: 0,00 % от начальной (максимальной) цены контракта, что составляет 0,00 рублей.
----------------
Размер обеспечения исполнения контракта (договора) составляет: 0,50% от цены
------------------------------------------------
Обеспечение исполнения Контракта устанавливается в размере 5 % от цены, по которой заключается Контракт, что составляет __________руб. _____коп.
----------------
Обеспечение исполнения Контракта устанавливается в размере 5 % от цены, по которой заключается Контракт, что составляет __
----------------------------------------------

KeyboardInterrupt: 

In [148]:
test_df = pd.read_json('/Users/damirabdulaev/Downloads/nlp_test_task_2023/dataset/test.json')
test_df

,id,text,label
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта
...,...,...,...
313,854936033,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,обеспечение гарантийных обязательств
314,576390745,Часть III Проект договора Договор №______ пост...,обеспечение гарантийных обязательств
315,323745820,УТВЕРЖДАЮ Председатель единой комиссии по осущ...,обеспечение гарантийных обязательств
316,712286194,Версия с 04.07.2022 года У Т В Е Р Ж Д А Ю «Го...,обеспечение гарантийных обязательств


In [149]:
# Define the function to extract the relevant part of the text
def extract_part(row):
    # Apply the NLP model to the text
    doc = nlp(row['text'])

    entity_names = []
    for ent in doc.ents:
        entity_names.append(ent.text)

    # Find the first sentence in the document
    if len(entity_names) == 0:
        sentence = ""
    else:
        sentence = entity_names[0]

    # Get the start and end indexes of the sentence in the document
    start = sentence.find(sentence)
    end = start + len(sentence)

    # Return a dictionary with the extracted part and its start and end indexes
    return {'text': sentence, 'answer_start': start, 'answer_end': end}

test_df['extracted_part'] = test_df.apply(extract_part, axis=1)


In [150]:
test_df

,id,text,label,extracted_part
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта,{'text': 'Размер обеспечения исполнения Контра...
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': 'Размер обеспечения исполнения контра...
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта,{'text': 'Размер обеспечение исполнения контра...
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта,{'text': 'Размер обеспечение исполнения контра...
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта,{'text': 'Размер обеспечение исполнения контра...
...,...,...,...,...
313,854936033,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯ...,обеспечение гарантийных обязательств,"{'text': '', 'answer_start': 0, 'answer_end': 0}"
314,576390745,Часть III Проект договора Договор №______ пост...,обеспечение гарантийных обязательств,{'text': 'Размер обеспечения исполнения догово...
315,323745820,УТВЕРЖДАЮ Председатель единой комиссии по осущ...,обеспечение гарантийных обязательств,"{'text': '', 'answer_start': 0, 'answer_end': 0}"
316,712286194,Версия с 04.07.2022 года У Т В Е Р Ж Д А Ю «Го...,обеспечение гарантийных обязательств,{'text': 'Обеспечение исполнения договора: 5% ...


In [151]:
np.savetxt('/Users/damirabdulaev/Desktop/Project/predictions.json', test_df.values, fmt='%s')